# LaFave and Thomas (2016) 
## Table 2 replication 
### Author: Grady Killeen
### Date: March 2022 

This notebook replicates Table 2 from LaFave and Thomas (2016) using Python. The goal of this notebook is to translate the replication code into open source software so that a broader audience can consider robustness checks and modifications. 

In [1]:
# Package and data imports 
import pandas as pd 
import numpy as np 
from linearmodels import PanelOLS
from linearmodels.iv.model import IV2SLS
from patsy import dmatrices
import pyhdfe
import statsmodels.api as sm
from IPython.display import display  # For cleaning displaying Pandas dataframes in the notebook

df = pd.read_stata('../LaFaveThomas.dta') 
df['obs'] = df.groupby('farmhhid').cumcount() + 1 # We need a second index for the PanelOLS specification even though obs is not used
df.set_index(['farmhhid', 'obs'], inplace=True)
display(df.head())


farmhhcommtime  labor_d_log  m0014  m1519  m2034  m3549  m5064  \
farmhhid obs                                                                   
1        1              1171     4.056989      1      0      0      1      0   
         2               500     5.113192      1      0      0      1      0   
         3              1522     4.707727      1      1      0      1      0   
         4              1201     2.990720      1      0      0      1      0   
         5               251     3.190476      1      0      0      1      0   

              mge65  f0014  f1519  ...  m_exist  f_exist  ivwmth  agingonly  \
farmhhid obs                       ...                                        
1        1        0      2      1  ...        1        1       9          0   
         2        0      3      0  ...        1        1       3          0   
         3        0      2      0  ...        1        1       5          0   
         4        0      3      0  ...        1        1       8          0   
         5        0      2      1  ...        1        1      10          0   

              edyrs_strat  bottom50  top50  bottom15  mid70  top15  
farmhhid obs                                                        
1        1              2         1      0         1      0      0  
         2              2         1      0         1      0      0  
         3              2         1      0         1      0      0  
         4              2         1      0         1      0      0  
         5              2         1      0         1      0      0  

[5 rows x 93 columns]

Next, we turn to estimating the regressions. We use the same specifications as the replication files from LaFave and Thomas (2016). See the paper for more details. 

The linear models package is used for estimating the panel regressions. A Wald test is used for the post-estimation tests. This returns a $\chi^2_q$ distributed statistic, so it's divided by the degrees of freedom to obtain an F-stat comparable with Stata's test command. Coefficient estimates are essentially identical. The F-stats are similar, but very slightly. 

For the fixed-effects regressions, the sample size varies slightly but coefficient estimates seem to be approximately indentical. F-stats are somewhat different. They very slightly more for fixed-effects regressions, although p-values are very similar.


In [2]:
# Define a vector with household composition variables: preferred linear model and Deaton shares model
hhcomp = 'm0014 + m1519 + m2034 + m3549 + m5064 + mge65 + f0014 + f1519 + f2034 + f3549 + f5064 + fge65'
hhcomp_list = hhcomp.split(' + ')

# Define a similar vector of household shares variables 
hhshares = 'hhsize_log + sm1519 + sm2034 + sm3549 + sm5064 + smge65 + sf0014 + sf1519 + sf2034 + sf3549 + sf5064 + sfge65'
hhshares_list = hhshares.split(' + ')

# Define a vector of RHS control variables 
rhs = 'C(farmass_q)  + C(hhass_q) + m_age + m_educ + m_exist + f_age + f_educ + f_exist + C(ivwmth)'
rhs_list = ['farmass_q', 'hhass_q', 'm_age', 'm_educ', 'm_exist', 'f_age', 'f_educ', 'f_exist', 'ivwmth']
rhs_list_no_factors = ['m_age', 'm_educ', 'm_exist', 'f_age', 'f_educ', 'f_exist']
rhs_list_factors = ['farmass_q', 'hhass_q', 'ivwmth']

# Define the Wald tests performed post-regression 
wt1 = 'm0014 = m1519 = m2034 = m3549 = m5064 = mge65 = f0014 = f1519 = f2034 = f3549 = f5064 = fge65 = 0'
wt2 = 'm0014 = m1519 = m2034 = m3549 = m5064 = mge65 = 0'
wt3 = 'f0014 = f1519 = f2034 = f3549 = f5064 = fge65 = 0'
wt4 = 'm1519 = m2034 = m3549 = f1519 = f2034 = f3549 = 0'

# In terms of shares 
swt1 = 'hhsize_log = sm1519 = sm2034 = sm3549 = sm5064 = smge65 = sf0014 = sf1519 = sf2034 = sf3549 = sf5064 = sfge65 = 0'
swt2 = 'sm1519 = sm2034 = sm3549 = sm5064 = smge65 = 0'
swt3 = 'sf0014 = sf1519 = sf2034 = sf3549 = sf5064 = sfge65 = 0'
swt4 = 'sm1519 = sm2034 = sm3549 = sf1519 = sf2034 = sf3549 = 0'
wt_index = ['All groups', 'Males', 'Females', 'Prime age adults']

# Table 2 column 1 
print('Column 1')
model = 'labor_d_log ~ 1 + ' + hhcomp + " + " + rhs 

column1 = PanelOLS.from_formula(model, data=df, other_effects=df['farmhhcommtime']).fit(cov_type="clustered", cluster_entity=True)

print(column1.summary)

print('Post estimation tests')

c1_tests = np.empty((4, 2))

test1 = column1.wald_test(formula=wt1)
# The linear models package reports a chi-squared test statistic. To get the equivalent f-stat, we divide by df
# The Chi squared and F tail probabilities are about identical, so we just use the reported p-value
c1_tests[0, :] = [test1.stat/test1.df, test1.pval]
test2 = column1.wald_test(formula=wt2)
c1_tests[1, :] = [test2.stat/test2.df, test2.pval]
test3 = column1.wald_test(formula=wt3)
c1_tests[2, :] = [test3.stat/test3.df, test3.pval]
test4 = column1.wald_test(formula=wt4)
c1_tests[3, :] = [test4.stat/test4.df, test4.pval]

c1_tests = pd.DataFrame(data=c1_tests, columns=['F-stat', 'p'], index=wt_index)
display(c1_tests)

# Column 2
print('Column 2')
model = 'labor_d_log ~ 1 + ' + hhshares + " + " + rhs 
column2 = PanelOLS.from_formula(model, data=df, other_effects=df['farmhhcommtime']).fit(cov_type="clustered", cluster_entity=True)

print(column2.summary)

print('Post estimation tests')

c2_tests = np.empty((4, 2))
test1 = column2.wald_test(formula=swt1)
c2_tests[0, :] = [test1.stat/test1.df, test1.pval]
test2 = column2.wald_test(formula=swt2)
c2_tests[1, :] = [test2.stat/test2.df, test2.pval]
test3 = column2.wald_test(formula=swt3)
c2_tests[2, :] = [test3.stat/test3.df, test3.pval]
test4 = column2.wald_test(formula=swt4)
c2_tests[3, :] = [test4.stat/test4.df, test4.pval]

c2_tests = pd.DataFrame(data=c2_tests, columns=['F-stat', 'p'], index=wt_index)
display(c2_tests)

# Table 2 Column 3
print('Column 3')
model = 'labor_d_log ~ 1 + ' + hhcomp + " + " + rhs + ' + EntityEffects'
column3 = PanelOLS.from_formula(model, data=df, other_effects=df['farmhhcommtime']).fit(use_lsmr=True, cov_type="clustered", cluster_entity=True)

print(column3.summary)

print('Post estimation tests')

c3_tests = np.empty((4, 2))
test1 = column3.wald_test(formula=wt1)
c3_tests[0, :] = [test1.stat/test1.df, test1.pval]
test2 = column3.wald_test(formula=wt2)
c3_tests[1, :] = [test2.stat/test2.df, test2.pval]
test3 = column3.wald_test(formula=wt3)
c3_tests[2, :] = [test3.stat/test3.df, test3.pval]
test4 = column3.wald_test(formula=wt4)
c3_tests[3, :] = [test4.stat/test4.df, test4.pval]

c3_tests = pd.DataFrame(data=c3_tests, columns=['F-stat', 'p'], index=wt_index)
display(c3_tests)

# Table 2 Column 4
print('Column 4')

# Get aging only sample 
aging = df.loc[df['agingonly'] == 1, :]

# Redefine demographic specification to include reference groups (birth-14). Total number of males 
# and total n. of females is constant over time and picked up by hhFE 
hhcomp_aging = 'm1519 + m2034 + m3549 + m5064 + mge65 + f1519 + f2034 + f3549 + f5064 + fge65'
hhcomp_aging_list = hhcomp.split(' + ')
wt1_aging = 'm1519 = m2034 = m3549 = m5064 = mge65 = f1519 = f2034 = f3549 = f5064 = fge65 = 0'
wt2_aging = 'm1519 = m2034 = m3549 = m5064 = mge65 = 0'
wt3_aging = 'f1519 = f2034 = f3549 = f5064 = fge65 = 0'

model = 'labor_d_log ~ 1 + ' + hhcomp_aging + " + " + rhs + ' + EntityEffects'

column4 = PanelOLS.from_formula(model, data=aging, other_effects=aging['farmhhcommtime']).fit(use_lsmr=True, cov_type="clustered", cluster_entity=True)

print(column4.summary)

print('Post estimation tests')

c4_tests = np.empty((4, 2))
test1 = column4.wald_test(formula=wt1_aging)
c4_tests[0, :] = [test1.stat/test1.df, test1.pval]
test2 = column4.wald_test(formula=wt2_aging)
c4_tests[1, :] = [test2.stat/test2.df, test2.pval]
test3 = column4.wald_test(formula=wt3_aging)
c4_tests[2, :] = [test3.stat/test3.df, test3.pval]
test4 = column4.wald_test(formula=wt4)
c4_tests[3, :] = [test4.stat/test4.df, test4.pval]

c4_tests = pd.DataFrame(data=c4_tests, columns=['F-stat', 'p'], index=wt_index)
display(c4_tests)

# Table 2 Column 5
print('Column 5')

# Formulas with lagged values 
hhcomp_l = 'm0014_l1 + m1519_l1 + m2034_l1 + m3549_l1 + m5064_l1 + mge65_l1 + f0014_l1 + f1519_l1 + f2034_l1 + f3549_l1 + f5064_l1 + fge65_l1'
hhcomp_l_list = hhcomp_l.split(' + ')

wt1_l = 'm1519_l1 = m2034_l1 = m3549_l1 = m5064_l1 = mge65_l1 = f1519_l1 = f2034_l1 = f3549_l1 = f5064_l1 = fge65_l1 = 0'
wt2_l = 'm0014_l1 = m1519_l1 = m2034_l1 = m3549_l1 = m5064_l1 = mge65_l1 = 0'
wt3_l = 'f0014_l1 = f1519_l1 = f2034_l1 = f3549_l1 = f5064_l1 = fge65_l1 = 0'
wt4_l = 'm1519_l1 = m2034_l1 = m3549_l1 = f1519_l1 = f2034_l1 = f3549_l1 = 0'

model = 'labor_d_log ~ 1 + ' + hhcomp_l + " + " + rhs + ' + EntityEffects'

column5 = PanelOLS.from_formula(model, data=df, other_effects=df['farmhhcommtime']).fit(use_lsmr=True, cov_type="clustered", cluster_entity=True)

print(column5.summary)

print('Post estimation tests')

c5_tests = np.empty((4, 2))
test1 = column5.wald_test(formula=wt1_l)
c5_tests[0, :] = [test1.stat/test1.df, test1.pval]
test2 = column5.wald_test(formula=wt2_l)
c5_tests[1, :] = [test2.stat/test2.df, test2.pval]
test3 = column5.wald_test(formula=wt3_l)
c5_tests[2, :] = [test3.stat/test3.df, test3.pval]
test4 = column5.wald_test(formula=wt4_l)
c5_tests[3, :] = [test4.stat/test4.df, test4.pval]

c5_tests = pd.DataFrame(data=c5_tests, columns=['F-stat', 'p'], index=wt_index)
display(c5_tests)

# Table 2 Column 6
print('Column 6')

# Formulas with lagged values 
hhcomp_f = 'm0014_f1 + m1519_f1 + m2034_f1 + m3549_f1 + m5064_f1 + mge65_f1 + f0014_f1 + f1519_f1 + f2034_f1 + f3549_f1 + f5064_f1 + fge65_f1'
hhcomp_f_fist = hhcomp_f.split(' + ')

wt1_f = 'm1519_f1 = m2034_f1 = m3549_f1 = m5064_f1 = mge65_f1 = f1519_f1 = f2034_f1 = f3549_f1 = f5064_f1 = fge65_f1 = 0'
wt2_f = 'm0014_f1 = m1519_f1 = m2034_f1 = m3549_f1 = m5064_f1 = mge65_f1 = 0'
wt3_f = 'f0014_f1 = f1519_f1 = f2034_f1 = f3549_f1 = f5064_f1 = fge65_f1 = 0'
wt4_f = 'm1519_f1 = m2034_f1 = m3549_f1 = f1519_f1 = f2034_f1 = f3549_f1 = 0'

model = 'labor_d_log ~ 1 + ' + hhcomp_f + " + " + rhs + ' + EntityEffects'

column6 = PanelOLS.from_formula(model, data=df, other_effects=df['farmhhcommtime']).fit(use_lsmr=True, cov_type="clustered", cluster_entity=True)

print(column6.summary)

print('Post estimation tests')

c6_tests = np.empty((4, 2))
test1 = column6.wald_test(formula=wt1_f)
c6_tests[0, :] = [test1.stat/test1.df, test1.pval]
test2 = column6.wald_test(formula=wt2_f)
c6_tests[1, :] = [test2.stat/test2.df, test2.pval]
test3 = column6.wald_test(formula=wt3_f)
c6_tests[2, :] = [test3.stat/test3.df, test3.pval]
test4 = column6.wald_test(formula=wt4_f)
c6_tests[3, :] = [test4.stat/test4.df, test4.pval]

c6_tests = pd.DataFrame(data=c6_tests, columns=['F-stat', 'p'], index=wt_index)
display(c6_tests)

# Column 7 
ivs = 'm0014_l1 + m1519_l1 + m2034_l1 + m3549_l1 + m5064_l1 + mge65_l1 + f0014_l1 + f1519_l1 + f2034_l1 + f3549_l1 + f5064_l1 + fge65_l1 + m0014_l2 + m1519_l2 + m2034_l2 + m3549_l2 + m5064_l2 + mge65_l2 + f0014_l2 + f1519_l2 + f2034_l2 + f3549_l2 + f5064_l2 + fge65_l2 + m0014_l3 + m1519_l3 + m2034_l3 + m3549_l3 + m5064_l3 + mge65_l3 + f0014_l3 + f1519_l3 + f2034_l3 + f3549_l3 + f5064_l3 + fge65_l3'
ivs_list = ivs.split(' + ')
c7_vars = ivs_list + hhcomp_list + rhs_list + ['farmhhcommtime', 'labor_d_log']
temp = df[c7_vars]
temp = temp.copy(deep=True)  # To avoid changing original data
temp.dropna(inplace=True)
temp.reset_index(inplace=True)

# Drop singletons 
temp = temp.groupby('farmhhcommtime').filter(lambda x: len(x) > 1)
temp = temp.groupby('farmhhid').filter(lambda x: len(x) > 1)

# Get design matrices 
second_model = 'labor_d_log ~ -1 + ' + hhcomp 
rhs_model = 'labor_d_log ~ -1 + ' + rhs  # Outcome irrelevant here, want x matrix to demean for IVs
first_model = 'labor_d_log ~ -1 + ' + ivs  # Outcome irrelevant here, just want x matrix to demean for IVs
y_df, x_df = dmatrices(second_model, temp, 1, NA_action='raise', return_type='dataframe')
temp_df, rhs_df = dmatrices(rhs_model, temp, 1, NA_action='raise', return_type='dataframe')
_y_df, iv_df = dmatrices(first_model, temp, 1, NA_action='raise', return_type='dataframe')

to_absorb = temp[['farmhhcommtime', 'farmhhid']].to_numpy()
algorithm = pyhdfe.create(to_absorb, drop_singletons=False, compute_degrees=False)

# Absorb fixed effects
_y = np.reshape(y_df.to_numpy(), (len(y_df), 1))
y_resid = algorithm.residualize(_y)
x_resid = algorithm.residualize(x_df.to_numpy())
rhs_resid = algorithm.residualize(rhs_df.to_numpy())
iv_resid = algorithm.residualize(iv_df.to_numpy())

y_df = pd.DataFrame(data=y_resid, columns=['labor_d_log'])
x_df = pd.DataFrame(data=x_resid, columns=x_df.columns)
rhs_df = pd.DataFrame(data=rhs_resid, columns=rhs_df.columns)
iv_df = pd.DataFrame(data=iv_resid, columns=iv_df.columns)

rhs_df = rhs_df.drop(columns=['C(farmass_q)[1]'])  # This should have been dropped, ensures full column rank

column7 = IV2SLS(y_df, rhs_df, x_df, iv_df).fit(cov_type='clustered', clusters=temp['farmhhid'])
print(column7.summary)

j_stat = column7.sargan.stat  # Sargan-Hansen j-stat
j_stat_p = column7.sargan.pval

c7_tests = np.empty((4, 2))

test1 = column7.wald_test(formula=wt1)
c7_tests[0, :] = [test1.stat/test1.df, test1.pval]
test2 = column7.wald_test(formula=wt2)
c7_tests[1, :] = [test2.stat/test2.df, test2.pval]
test3 = column7.wald_test(formula=wt3)
c7_tests[2, :] = [test3.stat/test3.df, test3.pval]
test4 = column7.wald_test(formula=wt4)
c7_tests[3, :] = [test4.stat/test4.df, test4.pval]

c7_tests = pd.DataFrame(data=c7_tests, columns=['F-stat', 'p'], index=wt_index)
display(c7_tests)

# Column 8 
print('Column 8')
model = 'labor_d_llvd_log ~ 1 + ' + hhcomp + " + " + rhs + ' + EntityEffects'
column8 = PanelOLS.from_formula(model, data=df, other_effects=df['farmhhcommtime']).fit(use_lsmr=True, cov_type="clustered", cluster_entity=True)

print(column8.summary)

print('Post estimation tests')

c8_tests = np.empty((4, 2))
test1 = column8.wald_test(formula=wt1)
c8_tests[0, :] = [test1.stat/test1.df, test1.pval]
test2 = column8.wald_test(formula=wt2)
c8_tests[1, :] = [test2.stat/test2.df, test2.pval]
test3 = column8.wald_test(formula=wt3)
c8_tests[2, :] = [test3.stat/test3.df, test3.pval]
test4 = column8.wald_test(formula=wt4)
c8_tests[3, :] = [test4.stat/test4.df, test4.pval]

c8_tests = pd.DataFrame(data=c8_tests, columns=['F-stat', 'p'], index=wt_index)
display(c8_tests)

# Column 9
print('Column 9')
model = 'labor_d_wpf_log ~ 1 + ' + hhcomp + " + " + rhs + ' + EntityEffects'
column9 = PanelOLS.from_formula(model, data=df, other_effects=df['farmhhcommtime']).fit(use_lsmr=True, cov_type="clustered", cluster_entity=True)

print(column9.summary)

print('Post estimation tests')

c9_tests = np.empty((4, 2))
test1 = column9.wald_test(formula=wt1)
c9_tests[0, :] = [test1.stat/test1.df, test1.pval]
test2 = column9.wald_test(formula=wt2)
c9_tests[1, :] = [test2.stat/test2.df, test2.pval]
test3 = column9.wald_test(formula=wt3)
c9_tests[2, :] = [test3.stat/test3.df, test3.pval]
test4 = column9.wald_test(formula=wt4)
c9_tests[3, :] = [test4.stat/test4.df, test4.pval]

c9_tests = pd.DataFrame(data=c9_tests, columns=['F-stat', 'p'], index=wt_index)
display(c9_tests)

# Column 10
print('Column 10')
model = 'labor_d_h_log ~ 1 + ' + hhcomp + " + " + rhs + ' + EntityEffects'
column10 = PanelOLS.from_formula(model, data=df, other_effects=df['farmhhcommtime']).fit(use_lsmr=True, cov_type="clustered", cluster_entity=True)

print(column10.summary)

print('Post estimation tests')

c10_tests = np.empty((4, 2))
test1 = column10.wald_test(formula=wt1)
c10_tests[0, :] = [test1.stat/test1.df, test1.pval]
test2 = column10.wald_test(formula=wt2)
c10_tests[1, :] = [test2.stat/test2.df, test2.pval]
test3 = column10.wald_test(formula=wt3)
c10_tests[2, :] = [test3.stat/test3.df, test3.pval]
test4 = column10.wald_test(formula=wt4)
c10_tests[3, :] = [test4.stat/test4.df, test4.pval]

c10_tests = pd.DataFrame(data=c10_tests, columns=['F-stat', 'p'], index=wt_index)
display(c10_tests)

Column 1
                          PanelOLS Estimation Summary                           
Dep. Variable:            labor_d_log   R-squared:                        0.1849
Estimator:                   PanelOLS   R-squared (Between):              0.2553
No. Observations:               38189   R-squared (Within):               0.0271
Date:                Wed, Mar 23 2022   R-squared (Overall):              0.1892
Time:                        17:42:48   Log-likelihood                -4.333e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      223.88
Entities:                        4452   P-value                           0.0000
Avg Obs:                       8.5779   Distribution:                F(37,36516)
Min Obs:                       1.0000                                           
Max Obs:                       11.000   F-statistic (robust):             88.009
                   

,F-stat,p
All groups,37.220981,0.000000e+00
Males,49.816704,0.000000e+00
Females,10.567422,9.128809e-12
Prime age adults,45.069617,0.000000e+00


Column 2
                          PanelOLS Estimation Summary                           
Dep. Variable:            labor_d_log   R-squared:                        0.1828
Estimator:                   PanelOLS   R-squared (Between):              0.2544
No. Observations:               38189   R-squared (Within):               0.0277
Date:                Wed, Mar 23 2022   R-squared (Overall):              0.1882
Time:                        17:42:49   Log-likelihood                -4.338e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      220.82
Entities:                        4452   P-value                           0.0000
Avg Obs:                       8.5779   Distribution:                F(37,36516)
Min Obs:                       1.0000                                           
Max Obs:                       11.000   F-statistic (robust):             86.189
                   

,F-stat,p
All groups,33.609486,0.000000e+00
Males,21.641736,0.000000e+00
Females,10.976625,2.879030e-12
Prime age adults,14.532449,1.110223e-16


Column 3
                          PanelOLS Estimation Summary                           
Dep. Variable:            labor_d_log   R-squared:                        0.0328
Estimator:                   PanelOLS   R-squared (Between):              0.1239
No. Observations:               38189   R-squared (Within):               0.0661
Date:                Wed, Mar 23 2022   R-squared (Overall):              0.1393
Time:                        17:42:56   Log-likelihood                 -3.14e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      29.430
Entities:                        4452   P-value                           0.0000
Avg Obs:                       8.5779   Distribution:                F(37,32065)
Min Obs:                       1.0000                                           
Max Obs:                       11.000   F-statistic (robust):             19.936
                   

...

,F-stat,p
All groups,4.681798,1.110197e-07
Males,5.896109,3.642588e-06
Females,3.329597,2.794953e-03
Prime age adults,8.414497,3.754353e-09


Column 10


/home/grady/miniconda3/envs/econ270b/lib/python3.7/site-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          PanelOLS Estimation Summary                           
Dep. Variable:          labor_d_h_log   R-squared:                        0.0092
Estimator:                   PanelOLS   R-squared (Between):              0.0657
No. Observations:               24353   R-squared (Within):              -0.0015
Date:                Wed, Mar 23 2022   R-squared (Overall):              0.0451
Time:                        17:48:08   Log-likelihood                -2.513e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      4.7167
Entities:                        4022   P-value                           0.0000
Avg Obs:                       6.0549   Distribution:                F(37,18753)
Min Obs:                       1.0000                                           
Max Obs:                       11.000   F-statistic (robust):             3.5644
                            

,F-stat,p
All groups,4.019118,2.850995e-06
Males,5.448206,1.203271e-05
Females,1.490706,1.767385e-01
Prime age adults,6.451412,8.163734e-07


So we are able to replicate all results in Table 2 closely, although some of the postestimation test statistics and thus p-values differ. This is likely due to minor differences in the way the tests are calculated. However, the results are qualitatively the same, and quantitatively very similar. This thus seems in line with small expected differences. 

It's possible to output the results to LaTex in several ways. Perhaps the easiest is to create a Pandas DataFrame to store the results, which can then be exported. Since there are multiple examples of this elsewhere and it's not essential to the replication, this is omitted here. 

# Robustness 

One potential concern with the LaFave and Thomas results is that much of the variation reflects movement in one's life cycle. That is, as the farmer gets older, their children age into working age. So the variation analyzed earlier is potentially correlated with greater experience farming. If experience farming improves managerial skill, it could increase returns to labor (e.g. in Cobb-Douglas higher TFP increases labor demand). So this could confound. 

To test whether the results are robust to this, I consider variation from aging out of working age only. In particular, I estimate the household fixed-effects model on the number of men and women under 65 and under 50. Hence, variation is primarily driven by people aging out of working age. We would expect labor demand to be positively correlated with the number of workers under these ages if the results hold. I now test this. 

In [3]:
df['men_under_50'] = df['m0014'] + df['m1519'] + df['m2034'] + df['m3549'] 
df['men_under_65'] = df['men_under_50'] + df['m5064']

df['women_under_50'] = df['f0014'] + df['f1519'] + df['f2034'] + df['f3549'] 
df['women_under_65'] = df['women_under_50'] + df['f5064']

model = 'labor_d_log ~ men_under_50 + men_under_65 + women_under_50 + women_under_65 + ' + rhs

robustness = PanelOLS.from_formula(model, data=df, other_effects=df['farmhhcommtime']).fit(use_lsmr=True, cov_type="clustered", cluster_entity=True)
print(robustness.summary)

test_all = 'men_under_65 = men_under_50 = women_under_65 = women_under_50 = 0'
test_men = 'men_under_65 = men_under_50 = 0'
test_women = 'women_under_65 = women_under_50 = 0'

robustness_tests = np.empty((3, 2))
test1 = robustness.wald_test(formula=test_all)
robustness_tests[0, :] = [test1.stat/test1.df, test1.pval]
test2 = robustness.wald_test(formula=test_men)
robustness_tests[1, :] = [test2.stat/test2.df, test2.pval]
test3 = robustness.wald_test(formula=test_women)
robustness_tests[2, :] = [test3.stat/test3.df, test3.pval]

robustness_tests = pd.DataFrame(data=robustness_tests, columns=['F-stat', 'p'], index=['All', 'Men', 'Women'])
display(robustness_tests)


                          PanelOLS Estimation Summary                           
Dep. Variable:            labor_d_log   R-squared:                        0.1743
Estimator:                   PanelOLS   R-squared (Between):              0.2464
No. Observations:               38189   R-squared (Within):               0.0217
Date:                Wed, Mar 23 2022   R-squared (Overall):              0.1802
Time:                        17:48:08   Log-likelihood                -4.358e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      265.89
Entities:                        4452   P-value                           0.0000
Avg Obs:                       8.5779   Distribution:                F(29,36524)
Min Obs:                       1.0000                                           
Max Obs:                       11.000   F-statistic (robust):             7347.2
                            

,F-stat,p
All,68.968747,0.000000e+00
Men,105.669985,0.000000e+00
Women,17.197702,3.397294e-08


We see that the demographic variables significantly predict labor demand, so the results are robust to this speficication. 